In [2]:
import os
import psycopg2
import redshift_connector
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
postgresqlconn_data = dict(
    host = os.environ["HOST"],
    port = os.environ["PORT"],
    user = os.environ["USER"],
    password = os.environ["PASS"],
    database = os.environ["DB"]
)

In [12]:
postgresql_conn = psycopg2.connect(**postgresqlconn_data)
postgresql_cur = postgresql_conn.cursor()

In [14]:
query = """
select
	id
	, raw_data::varchar
	, CAN_JSON_PARSE(raw_data::VARCHAR) as can_json
	, JSON_SERIALIZE(raw_data) as json
	, JSON_EXTRACT_PATH_TEXT(raw_data::varchar, 'llave') as extraccion
	, JSON_EXTRACT_PATH_TEXT(raw_data::varchar, 'llave_inexistente') as inexistente
from adp_dwh.co_sandbox_datos.tmp_test_fabio
;
"""

postgresql_cur.execute(query)
postgresql_data = postgresql_cur.fetchall()
postgresql_data

[(1, None, None, 'null', None, None),
 (1, '{"llave": "Valor"}', True, '"{\\"llave\\": \\"Valor\\"}"', 'Valor', '')]

In [19]:
from src.processor import PostgreSQLProcessor, RedshiftProcessor

In [20]:
postgresqlconn_data = dict(
    host = os.environ["HOST"],
    port = os.environ["PORT"],
    user = os.environ["USER"],
    password = os.environ["PASS"],
    database = os.environ["DB"]
)

processor = PostgreSQLProcessor(**postgresqlconn_data)
processor.connect()


In [21]:
query = """
select
	id
	, raw_data::varchar
	, CAN_JSON_PARSE(raw_data::VARCHAR) as can_json
	, JSON_SERIALIZE(raw_data) as json
	, JSON_EXTRACT_PATH_TEXT(raw_data::varchar, 'llave') as extraccion
	, JSON_EXTRACT_PATH_TEXT(raw_data::varchar, 'llave_inexistente') as inexistente
from adp_dwh.co_sandbox_datos.tmp_test_fabio
;
"""

processor.fetch_data_as_dataframe(query)

/mnt/N1662770/poc_soda_files/src/processor.py:98: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, self.connection)


,id,raw_data,can_json,json,extraccion,inexistente
0,1,None,None,null,None,None
1,1,"{""llave"": ""Valor""}",True,"""{\""llave\"": \""Valor\""}""",Valor,
